In [ ]:
import pandas as pd
from pathlib import Path
import json

from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = None

In [2]:
try:
    # for pyhton file .py
    PATH = Path(__file__).resolve().parent.parent
except NameError:
    # For Jupyter Notebook
    PATH = Path().resolve().parent.parent

with open(PATH / "config.json", "r") as f:
    VARIABLES = json.load(f)

# path of the folder with .parquet
folder_path = Path(VARIABLES["file_path_box_builder_dataset_CETIC"])

In [ ]:
feature_frame = pd.read_csv(folder_path)
feature_frame
#copy, for if necesary reload the dataset
feature_frame_org=feature_frame[:]

In [ ]:
feature_frame = feature_frame_org

In [67]:
feature_frame=feature_frame[feature_frame["user_order_seq"]>=5]
feature_frame.head(5)

,variant_id,product_type,order_id,user_id,created_at,order_date,user_order_seq,outcome,ordered_before,abandoned_before,active_snoozed,set_as_regular,normalised_price,discount_pct,vendor,global_popularity,count_adults,count_children,count_babies,count_pets,people_ex_baby,days_since_purchase_variant_id,avg_days_to_buy_variant_id,std_days_to_buy_variant_id,days_since_purchase_product_type,avg_days_to_buy_product_type,std_days_to_buy_product_type
6,33826472919172,ricepastapulses,2808548917380,3476645445764,2020-10-06 14:23:08,2020-10-06,5,0.0,1.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.038462,2.0,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
7,33826472919172,ricepastapulses,2808549900420,3437017956484,2020-10-06 14:24:26,2020-10-06,13,0.0,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.038462,2.0,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
12,33826472919172,ricepastapulses,2809061539972,3437017956484,2020-10-07 08:48:22,2020-10-07,14,0.0,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.026316,2.0,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
13,33826472919172,ricepastapulses,2809064751236,3437017956484,2020-10-07 08:58:46,2020-10-07,15,0.0,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.026316,2.0,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
17,33826472919172,ricepastapulses,2809189335172,3223115595908,2020-10-07 13:54:54,2020-10-07,9,0.0,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.026316,2.0,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618


In [71]:
feature_frame[["days_since_purchase_variant_id","avg_days_to_buy_variant_id","std_days_to_buy_variant_id","days_since_purchase_product_type","avg_days_to_buy_product_type","std_days_to_buy_product_type"]].corr()

,days_since_purchase_variant_id,avg_days_to_buy_variant_id,std_days_to_buy_variant_id,days_since_purchase_product_type,avg_days_to_buy_product_type,std_days_to_buy_product_type
days_since_purchase_variant_id,1.000000,0.040183,0.047066,0.178246,0.010523,0.013018
avg_days_to_buy_variant_id,0.040183,1.000000,0.599918,0.008753,0.293421,0.251989
std_days_to_buy_variant_id,0.047066,0.599918,1.000000,0.008725,0.224434,0.255415
days_since_purchase_product_type,0.178246,0.008753,0.008725,1.000000,0.050448,0.053402
avg_days_to_buy_product_type,0.010523,0.293421,0.224434,0.050448,1.000000,0.838906
std_days_to_buy_product_type,0.013018,0.251989,0.255415,0.053402,0.838906,1.000000


In [45]:
(feature_frame["count_adults"] == 2).value_counts()

count_adults
True     447206
False     29396
Name: count, dtype: int64

In [40]:
feature_frame[["count_adults",	"count_children","count_babies","count_pets","people_ex_baby"]].corr()

,count_adults,count_children,count_babies,count_pets,people_ex_baby
count_adults,1.000000,0.040101,-0.023736,0.140629,0.619555
count_children,0.040101,1.000000,0.199633,0.555477,0.809166
count_babies,-0.023736,0.199633,1.000000,0.214941,0.142870
count_pets,0.140629,0.555477,0.214941,1.000000,0.519071
people_ex_baby,0.619555,0.809166,0.142870,0.519071,1.000000


# Columns
* variant id                            : -> Not include, inside of the model will be the caracteristics 
* product_type                          : -> One Hot encoding
* order_id                              :
* user_id                               :
* X created_at                          : are the same row, have the hour, it's innecesary -> *Delete*
* X order_date                          : There isn't any related to the time, I think it's innecesary here -> *Delete*
* ok user_order_seq                     :
* ok outcome                            :
* ok ordered_before                     :
* ok abandoned_before                   :
* ok active_snoozed                     :
* ok set_as_regular                     :
* normalised_price                      :
* discount_pct                          :
* vendor                                :
* ok global_popularity                  :
* count_adults                          :|
* count_children                        :|
* count_babies                          :|-Datos imputados
* count_pets                            :|
* people_ex_baby                        :|
* ok days_since_purchase_variant_id     :
* ok avg_days_to_buy_variant_id         :
* ok std_days_to_buy_variant_id         :
* ok days_since_purchase_product_type   :   
* ok avg_days_to_buy_product_type       :
* ok std_days_to_buy_product_type       :


KeyError: "None of ['order_date'] are in the columns"

### Product type

In [ ]:
#Product type: need to be categorical encoding
Product_type_one_hot = pd.get_dummies(feature_frame["product_type"])
Product_type_one_hot

array([49, 49, 49, ..., 33, 33, 33])